# 연합인포맥스 홈페이지 2014.01.01-2019.07.12 
# '금리' 검색 결과 뉴스 크롤링 (날짜, title, content)

In [10]:
import scrapy
from urllib import parse

class YonhapSpider(scrapy.Spider):
    name = 'yonhap'
    allowed_domains = ['news.einfomax.co.kr']
    start_urls=[]
    for i in range(4000):
        i=str(i)
        urls = 'https://www.yna.co.kr/search/index?query=%EA%B8%88%EB%A6%AC'
        start_urls.append(urls)

        def parse(self, response):
             
            for qu in response.css('ul'):
                link=qu.css('a::attr(href)').get()
                next_page = link
            #if next_page is not None:
                yield response.follow(next_page, callback=self.pse)    
            
        def pse(self, response):   
            yield {
                    'title': response.css('h1.tit-article::text').get(),
                    'content': response.css('div.article::text').getall(),
                    'date':response.css('div.share-info span.tt::text').getall()
                    }

# 전처리

In [ ]:
import pandas as pd
import numpy as np
import datetime
import re
yonhap=pd.read_csv('연합인포맥스_2011-2019.csv')
yonhap['press']=' 연합인포맥스'
yonhap.date=yonhap.date.str[4:14]
yonhap['date']=yonhap['date'].apply(lambda x : x[:4]+x[5:7]+x[8:])
yonhap['date']=pd.to_datetime(yonhap['date'])
sep = '(끝)'
yonhap['content']=yonhap['content'].apply(lambda x : x.lstrip('\r\n\t').rstrip('\r\n\t').rstrip('\r\n').rstrip('\t\t').rstrip('\r\n\r\n\t\t\r\n\t\t,\r\n\r\n\t\t\'').split(sep, 1)[0])
yonhap['content']=yonhap['content'].apply(lambda x :re.sub('\(서울=연합인포맥스\)','',x)).apply(lambda x :re.sub('\(뉴욕=연합인포맥스\)','',x))

In [ ]:
yonhap['content']=yonhap['content'].apply(lambda x :re.sub('.+[가-힣]{2} \=','',x))

In [ ]:
from ekonlpy.sentiment import MPCK
def text2ngram(text):
    mpck = MPCK()
    bef_tokens = mpck.tokenize(text)
    ngrams = mpck.ngramize(bef_tokens)
    return ngrams

In [ ]:
ngram=[]
for i in range(120036):
    strcat.(a[ngram][i],(text2ngram(yonhap['content'][i]))
yonhap['ngram']=ngram            

In [ ]:
yoninfo1= pd.read_csv("연합인포맥스 ngram 2005-2010.csv",index_col=0)
yoninfo1['date']=pd.to_datetime(yoninfo1['date'])

In [ ]:
yonhap_info=yoninfo1.append(yonhap)
yonhap_info= yonhap_info.reset_index(drop=True)
yonhap_info= yonhap_info.sort_values(by='date')
yonhap_info=yonhap_info.drop([140978,])
col_list=['date','press','title','content','ngram']
yonhap_info=yonhap_info[col_list]
yonhap_info.to_csv("yonhap_info_2005-2019_전처리.csv")

In [ ]:
import pandas as pd
a=pd.read_csv("(완료)연합인포맥스_2005-2019_전처리.csv")

In [ ]:
from ekonlpy.sentiment import MPCK
def text2ngram(text):
    mpck = MPCK()
    bef_tokens = mpck.tokenize(text)
    return bef_tokens

In [ ]:
for i in range(101,140979):    
    try:
        a['ngram'][i]=a['ngram'][i]+",".join(text2ngram(a['content'][i]))
    except:
        pass

In [12]:
# a.to_csv("(완료)연합인포맥스_2005-2019_전처리.csv")
call=pd.read_csv("callrate_ver5.csv", index_col=0)
call['date']=pd.to_datetime(call['date'])
a['date'] = pd.to_datetime(a['date'])
pd.merge(a,call,on='date',how='outer')

NameError: name 'a' is not defined

In [ ]:
yonhap_and_call=pd.merge(a,call,on='date',how='outer')
yonhap_and_call.to_csv("(완료)연합인포맥스_2005-2019_콜금리라벨까지.csv")

In [ ]:
import pandas as pd
yonhap= pd.read_csv("(완료)연합인포맥스_2005-2019_콜금리라벨까지.csv", index_col=0)
yonhap=yonhap.drop(['Unnamed: 0.1','updown','old_date','callrate','old_callrate','updown'],axis=1)
call= pd.read_csv("my_callrate_ffill.csv", index_col=0)
call['date']=pd.to_datetime(call['date'])
yonhap['date']=pd.to_datetime(yonhap['date'])
call=call.drop(['callrate','old_date','old_callrate'],axis=1)

In [ ]:
yonhap=pd.merge(yonhap,call,on='date',how='outer').sort_values(['date']).dropna().reset_index(drop=True)

In [ ]:
yonhap['n']=yonhap['ngram'].apply(lambda x : str(x).count(';'))
yonhap=yonhap[yonhap['n']>15]
yonhap['t/f']=yonhap['updown'].apply(lambda x : x in ('up','down','neutrality'))
yonhap=yonhap[yonhap['t/f']==True]
yonhap=yonhap.drop('t/f',axis=1)
down확률=len(yonhap[yonhap['updown']=='down'])/len(yonhap) 
up확률=len(yonhap[yonhap['updown']=='up'])/len(yonhap) 

In [ ]:
yonhap['ngram']=yonhap['ngram'].apply(lambda x:re.sub('[가-힣]{1,8}\/(JKS|JKC|JKG|JKO|JKB|JKV|JKQ|JC|JX|EP|EF|EC|ETN|ETM|XPN|XSN|XSV|XSA|XR|SF|SE|SSO|SSC|SC|SY|SH|SL|SN)(\,?)','',str(x)))

In [ ]:
yonhap.to_csv("(최종)연합인포맥스_2005-2019_콜금리라벨까지.csv")

In [ ]:
import pandas as pd
yonhap=pd.read_csv('(최종)연합인포맥스_2005-2019_콜금리라벨까지.csv',index_col=0)

In [ ]:
temp_ls = []

for i in range(len(yonhap)):
    try:
        test = yonhap['ngram'][i].replace(' ','').split(',') 
        temp_ls.extend(test) 
    except: 
        pass

temp_df = pd.DataFrame(temp_ls, columns=['ngram']) 
final_df = pd.DataFrame(temp_df['ngram'].value_counts())
final_df=final_df[final_df['ngram']>=15]
final_df['index1'] = final_df.index

In [ ]:
yonhap00=yonhap[yonhap['updown']=='up']
import pandas as pd
temp_l= []

for i in range(len(yonhap)):
    try:
        test = yonhap00['ngram'][i].replace(' ','').split(',') 
        temp_l.extend(test) 
    except: 
        pass

temp_dff = pd.DataFrame(temp_l, columns=['up_ngram']) 
final_dff = pd.DataFrame(temp_dff['up_ngram'].value_counts()) 
final_dff['index1'] = final_dff.index

In [ ]:
yonhap0=yonhap[yonhap['updown']=='down']
import pandas as pd
temp= []

for i in range(len(yonhap)):
    try:
        test = yonhap0['ngram'][i].replace(' ','').split(',') 
        temp.extend(test) 
    except: 
        pass

temp_d = pd.DataFrame(temp, columns=['down_ngram']) 
final_d = pd.DataFrame(temp_d['down_ngram'].value_counts()) 
final_d['index1'] = final_d.index

In [ ]:
up=pd.merge(final_df,final_dff,how='outer')
up['분자']=up['up_ngram']/len(temp_dff)

In [ ]:
down=pd.merge(final_df,final_d,how='outer')
down['분모']=down['down_ngram']/len(temp_d)
down.drop('ngram',axis=1)

In [ ]:
엄=pd.merge(up,down,how='outer',on='index1')
엄['polarity']=엄['분자']/엄['분모']
엄최종=엄[(엄['polarity']>1.3)|(엄['ploarity']<0.76)]
엄최종.reset_index(drop=True)
엄최종['hawk/dov']='hawkish'
엄최종['hawk/dov']=엄최종[엄['polarity']<0.76]['hawk/dov'].replace('hawkish','dovish')
엄최종.dropna()
엄최종['hawk/dov']=엄최종['hawk/dov'].fillna('hawkish')

In [ ]:
yonh=yonhap[yonhap['updown']=='neutrality']
import pandas as pd
temp= []

for i in range(len(yonhap)):
    try:
        test = yonh['ngram'][i].replace(' ','').split(',') 
        temp.extend(test) 
    except: 
        pass

temp_ = pd.DataFrame(temp, columns=['neu_ngram']) 
final_ = pd.DataFrame(temp_['neu_ngram'].value_counts()) 

# 연합뉴스 네이버 2005.05-2017.12.31 
# '금리' 검색 결과 뉴스 크롤링 (날짜, title, content)

In [ ]:
import scrapy
from datetime import timedelta, date
start_date = date(2005, 1, 1)
end_date = date(2005, 1, 16)
cnt_per_page = 10
keyword='�ݸ�'
url_format = "https://search.naver.com/search.naver?&where=news&query={1}&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={0}&de={0}&docid=&nso=so:r,p:from{0}to{0},a:all&mynews=1&cluster_rank=1&start={2}&refresh_start=0"

class YonhapSpider(scrapy.Spider):
    name = 'yonhap2'
    allowed_domains = ['naver.com','yna.co.kr']
    start_urls = []
    def daterange(start_date, end_date):
        for n in range(int ((end_date - start_date).days)):
            yield start_date + timedelta(n)
       
    for single_date in daterange(start_date, end_date):
        start_urls.append(url_format.format(single_date.strftime("%Y%m%d"),keyword, 1))
        
    def parse(self, response):    
        for qu in response.css('ul.type01'):
            link=qu.css('a::attr(href)').get()        
            next_page = link
            print(next_page)
    #         yield response.follow(next_page, callback=self.pse)
                    
    # def pse(self, response):   
    #     yield {
    #             'title': response.css('h1.tit-article::text').get(),
    #             'content': response.css('div.article::text').get(),
    #             'date':response.css('div.share-info span.tt::text').get()
    #           }


# 연합뉴스 2010-2014 전처리

In [4]:
import pandas as pd
import numpy as np
import datetime
import re
yon=pd.read_csv('연합뉴스_2010-2014.csv')
yon['press']=' 연합뉴스'
yon['date']=yon['date'].astype('str')
yon=yon.drop('link',axis=1)
yon['content'] = yon['content'].apply(lambda x: re.sub("\w*\@.+",'',x,1))
yon['content'] = yon['content'].apply(lambda x: re.sub("\([가-힣].=연합인포맥스\)|\([가-힣].=연합뉴스\)",'',x,1))
yon['date']=pd.to_datetime(yon['date'])
#yon['content']=yon['content'].apply(lambda x: re.sub(".+['기자']{2}\=|.+['기자']{2} \=",'',x,1))
# sep = '(끝)'
# yonhap['content']=yonhap['content'].apply(lambda x : x.lstrip('\r\n\t').rstrip('\r\n\t').rstrip('\r\n').rstrip('\t\t').rstrip('\r\n\r\n\t\t\r\n\t\t,\r\n\r\n\t\t\'').split(sep, 1)[0])
# yonhap['content']=yonhap['content'].apply(lambda x :re.sub('\(서울=연합인포맥스\)','',x)).apply(lambda x :re.sub('\(뉴욕=연합인포맥스\)','',x))

In [5]:
from ekonlpy.sentiment import MPCK
def text2ngram(text):
    mpck = MPCK()
    bef_tokens = mpck.tokenize(text)
    ngrams = mpck.ngramize(bef_tokens)
    return ngrams
n_gram=[]
for i in range(11732):
    n_gram.append(text2ngram(yon['content'][i]))
yon['ngram']=n_gram
yon.to_csv("yon.csv")

# 연합뉴스 2015,2016,2017 전처리

In [7]:
import pandas as pd
import numpy as np
import datetime
import re
yon1=pd.read_csv('연합뉴스_2015.csv')
yon1['press']='연합뉴스'
yon1=yon1.drop([1,])
yon1['date']=yon1['date'].astype('str')
yon1=yon1.drop('link',axis=1)
yon1['content'] = yon1['content'].apply(lambda x: re.sub("\w*\@.+",'',x,1))
yon1['content'] = yon1['content'].apply(lambda x: re.sub("\([가-힣].=연합인포맥스\)|\([가-힣].=연합뉴스\)",'',x,1))
yon1['date']=pd.to_datetime(yon1['date'])
yon1= yon1.reset_index(drop=True)

In [ ]:
yon1['content']=yon1['content'].apply(lambda x: re.sub('.+[가-힣]{2} \=|.+[가-힣]{2}\=','',x,1))

In [ ]:
from ekonlpy.sentiment import MPCK
def text2ngram(text):
    mpck = MPCK()
    bef_tokens = mpck.tokenize(text)
    ngrams = mpck.ngramize(bef_tokens)
    return ngrams

In [ ]:
n_gram=[]
for i in range(979):
    n_gram.append(text2ngram(yon1['content'][i]))

In [ ]:
yon1['ngram']=n_gram

In [ ]:
yon = pd.read_csv("yon.csv",index_col=0)
yon['date']=pd.to_datetime(yon['date'])
yon.head()

In [ ]:
yon2010_2015=yon.append(yon1)
yon2010_2015= yon2010_2015.reset_index(drop=True)
yon2010_2015.to_csv("연합뉴스2010_2015ngram.csv")

In [ ]:
import pandas as pd
import numpy as np
import datetime
import re
y2016['press']='연합뉴스'
# yon2=yon2.drop([1,])
y2016['date']=y2016['date'].astype('str')
y2016=y2016.drop('link',axis=1)
y2016['content'] = y2016['content'].apply(lambda x: re.sub("\w*\@.+",'',x,1))
y2016['content'] = y2016['content'].apply(lambda x: re.sub("\([가-힣].=연합인포맥스\)|\([가-힣].=연합뉴스\)",'',x,1))
y2016['date']=pd.to_datetime(y2016['date'])

In [ ]:
y=pd.read_csv("연합뉴스2010_2015ngram.csv",index_col=0)

In [ ]:
y2016=pd.read_csv("연합뉴스_2016.csv")

In [ ]:
from ekonlpy.sentiment import MPCK
def text2ngram(text):
    mpck = MPCK()
    bef_tokens = mpck.tokenize(text)
    ngrams = mpck.ngramize(bef_tokens)
    return ngrams

In [ ]:
n_gram=[]
for i in range(1313):
    n_gram.append(text2ngram(y2016['content'][i]))
y2016['ngram']=n_gram    

In [ ]:
y2017=pd.read_csv("연합뉴스_2017.csv")
y2017['press']='연합뉴스'
# y2017=y2017.drop([1,])
y2017['date']=y2017['date'].astype('str')
y2017=y2017.drop('link',axis=1)
y2017['content'] = y2017['content'].apply(lambda x: re.sub("\w*\@.+",'',x,1))
y2017['content'] = y2017['content'].apply(lambda x: re.sub("\([가-힣].=연합인포맥스\)|\([가-힣].=연합뉴스\)",'',x,1))
y2017['date']=pd.to_datetime(y2017['date'])

In [ ]:
n_gram=[]
for i in range(len(y2017['content'])):
    n_gram.append(text2ngram(y2017['content'][i]))
y2017['ngram']=n_gram    

In [ ]:
y2016_2017=y2016.append(y2017)
y2016_2017= y2016_2017.reset_index(drop=True)

In [ ]:
y2010_2015=pd.read_csv("연합뉴스2010_2015ngram.csv",index_col=0)
y2010_2015['date']=pd.to_datetime(y2010_2015['date'])

In [ ]:
y2010_2017=y2010_2015.append(y2016_2017)
y2010_2017=y2010_2017.reset_index(drop=True)

In [ ]:
y2005_2007=pd.read_csv("연합뉴스2005-2007.csv")
y2008_2009=pd.read_csv("연합뉴스2008-2009.csv")
y2005_2007['press']='연합뉴스'
y2005_2007['date']=y2005_2007['date'].astype('str')
y2005_2007=y2005_2007.drop('link',axis=1)
y2005_2007['content'] = y2005_2007['content'].apply(lambda x: re.sub("\w*\@.+",'',x,1))
y2005_2007['content'] = y2005_2007['content'].apply(lambda x: re.sub("\([가-힣].=연합인포맥스\)|\([가-힣].=연합뉴스\)",'',x,1))
y2005_2007['date']=pd.to_datetime(y2005_2007['date'])

In [8]:
y2008_2009['press']='연합뉴스'
y2008_2009['date']=y2008_2009['date'].astype('str')
y2008_2009=y2008_2009.drop('link',axis=1)
y2008_2009['content'] = y2008_2009['content'].apply(lambda x: re.sub("\w*\@.+",'',x,1))
y2008_2009['content'] = y2008_2009['content'].apply(lambda x: re.sub("\([가-힣].=연합인포맥스\)|\([가-힣].=연합뉴스\)",'',x,1))
y2008_2009['date']=pd.to_datetime(y2008_2009['date'])

NameError: name 'y2008_2009' is not defined

In [ ]:
n_gram=[]
for i in range(4432):
    n_gram.append(text2ngram(y2008_2009['content'][i]))
y2008_2009['ngram']=n_gram

In [ ]:
n_gram=[]
for i in range(5262):
    n_gram.append(text2ngram(y2005_2007['content'][i]))
y2005_2007['ngram']=n_gram
y2005_2007.head()

In [ ]:
y2005_2009=y2005_2007.append(y2008_2009)
y2005_2009=y2005_2009.reset_index(drop=True)
y2005_2017=y2005_2009.append(y2010_2017)
y2005_2017=y2005_2017.reset_index(drop=True)
y2005_2017.to_csv("(완료)연합뉴스2005-2017_ngram.csv")

# 의사록 어조

In [ ]:
import pandas as pd
import re
yonhap=pd.read_csv('(최종)모든자료date_ngram_label.csv',index_col=0)
yonhap['ngram'] = yonhap['ngram'].apply(lambda x:re.sub('[가-힣]{1,8}\/(JKS|JKC|JKG|JKO|JKB|JKV|JKQ|JC|JX|EP|EF|EC|ETN|ETM|XPN|XSN|XSV|XSA|XR|SF|SE|SSO|SSC|SC|SY|SH|SL|SN)(\,?)','',str(x),1))
yonhap['ngram'] = yonhap['ngram'].apply(lambda x: re.sub('\s+[가-힣]{1,8}\/(JKS|JKC|JKG|JKO|JKB|JKV|JKQ|JC|JX|EP|EF|EC|ETN|ETM|XPN|XSN|XSV|XSA|XR|SF|SE|SSO|SSC|SC|SY|SH|SL|SN)(\S*|\,)','',str(x),1))

In [ ]:
dr_rock_raw = pd.read_csv('(완성)_금통위_의사록.csv', encoding='utf-8', index_col=0)
dr_rock = dr_rock_raw.copy()

In [ ]:
our_polar = pd.read_csv('df_hawk_dov_new.csv', encoding='utf-8', index_col = 0)
our_polar = our_polar.reset_index(drop=False)
our_polar[our_polar.ngram.str.contains(';')][our_polar['hawk/dov']=='dovish']

In [ ]:
hawk = our_polar[our_polar['hawk/dov'] == 'hawkish']
hawk_list = list(hawk['ngram'])

In [ ]:
dov = our_polar[our_polar['hawk/dov'] == 'dovish']
dov = dov.reset_index(drop=True)
dov_list = list(dov['ngram'])

In [ ]:
temp_ls = [] # 빈리스트 생성
for i, row in dr_rock.iterrows(): # 사용할 데이터프레임의 길이로 for문 돌리기
    test = row['ngram'].split(',') # 컬럼이름 ngram의 i 번째를 split
    temp_ls.extend(test) # 빈 리스트에 한번에 추가
    if i%1000 == 0 : # %: 나머지, 1000개마다 프린트
        print('{}/{}'.format(i, dr_rock.shape[0]))
temp_df = pd.DataFrame(temp_ls, columns=['ngram']) # 새로운 데이터프레임 생성
df_up1 = pd.DataFrame(temp_df['ngram'].value_counts()) # value_counts사용
df_up1 = df_up1.reset_index()

In [ ]:
dr_rock['sentence_score'] = 0
dr_rock['sentence_score'] = dr_rock['sentence_score'].astype('float') 

for idx,val in enumerate(dr_rock['ngram']):
    word_list = val.replace(' ','').split(',')
    h_count = 0
    d_count = 0
    sentence_score = 0
    
    for i in word_list:
        if i in hawk_list:
            h_count += 1
            
        elif i in dov_list:
            d_count += 1
            
        else:
            continue
    
    bunja = float(h_count - d_count)
    bunmo = float(h_count + d_count)
    
    try:
        sentence_score = float(bunja / bunmo)
        dr_rock['sentence_score'][idx] = sentence_score
        
    except ZeroDivisionError:
        print("ZeroDivision")
        dr_rock['sentence_score'][idx] = 0
    
    
    if idx % 100 == 0 :
        print('현재 인덱스 : ',idx,'카운트 : ', sentence_score)
    

In [ ]:
dr_save = dr_rock.copy()
dr_save = dr_save.reset_index(drop=True)
dr_save[dr_save['date']=='2005-10-11']

In [ ]:
dr_score = pd.DataFrame(columns=['date', 'score'])
dr_score['score'] = dr_score['score'].astype('float')

for i in date_list:
    one_dr = dr_save[dr_save['date'] == i]
    pos_sentence = len(one_dr[one_dr['sentence_score'] >= 0])
    neg_sentence = len(one_dr[one_dr['sentence_score'] <0])
    
    
    bunmo = pos_sentence + neg_sentence
    bunja = pos_sentence - neg_sentence
    
    try:
        score = float(bunja / bunmo)
        dr_score = dr_score.append(pd.DataFrame([[i, score]], columns=['date','score']), ignore_index = True)
        
    except ZeroDivisionError:
        print("ZeroDivision")
        dr_score = dr_score.append(pd.DataFrame([[i, 0]], columns=['date','score']), ignore_index = True)

dr_score = dr_score.sort_values(['date'], ascending=[True])
dr_score= dr_score.reset_index(drop=True)
dr_score['date'] = pd.to_datetime(dr_score['date'])
dr_score = dr_score.set_index('date')

In [ ]:
date_list = list(set(dr_rock['date']))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(10, 5))
plt.plot(dr_score['score'])
plt.legend(loc=0)
plt.grid(True, color='0.7', linestyle=':', linewidth=1)
plt.ylim(-1,1)

In [ ]:
call_raw = pd.read_csv('기준금리2005_2019.csv', encoding='utf-8',index_col=0)
call_copy = call_raw.copy()
call_copy['date'] = pd.to_datetime(call_copy['date'])
call_copy = call_copy.sort_values(['date'], ascending=[True])
call_copy['BMR'] = (call_copy['BMR']-call_copy['BMR'].min())/(call_copy['BMR'].max()-call_copy['BMR'].min())
call_copy = call_copy.set_index('date')

plt.figure(figsize=(10, 5))
plt.plot(call_copy['BMR'])
plt.plot(dr_score['score'])
plt.legend(loc=0)
plt.grid(True, color='0.7', linestyle=':', linewidth=1)

In [ ]:
from sklearn import preprocessing
call_copy['BMR']=preprocessing.MinMaxScaler().fit_transform(call_copy['BMR'].values.reshape(-1,1))
dr_score['score']=preprocessing.MinMaxScaler().fit_transform(dr_score['score'].values.reshape(-1,1))
call_copy = call_copy.set_index('date')

In [ ]:
my_call_raw= pd.read_csv('my_callrate_ffill.csv', encoding='utf-8', index_col = 0)
my_call_copy = my_call_raw.copy()
my_call_copy['date'] = pd.to_datetime(my_call_copy['date'])
new_rate = my_call_copy.merge(call_copy, on ='date', how='left')
news_rate = new_rate.fillna(method = 'bfill', axis = 0)
news_rate = news_rate.drop('old_date', axis = 1).drop('callrate', axis = 1).drop('old_callrate', axis = 1).drop('updown', axis = 1)

In [14]:
from sklearn import preprocessing
news_rate['BMR']=preprocessing.MinMaxScaler().fit_transform(news_rate['BMR'].values.reshape(-1,1))
dr_score['score']=preprocessing.MinMaxScaler().fit_transform(dr_score['score'].values.reshape(-1,1))
news_rate['date'] = pd.to_datetime(news_rate['date'])
news_rate = news_rate.set_index('date')

NameError: name 'news_rate' is not defined

In [ ]:
news_rate['BMR'] = news_rate['BMR']-0.5
dr_score['score'] = dr_score['score']-0.5
news_rate['BMR'] = news_rate['BMR'] *2
dr_score['score'] = dr_score['score'] *2

plt.figure(figsize=(20, 10))
plt.plot(news_rate['BMR'])
plt.plot(dr_score['score'])
plt.legend(loc=0)
plt.grid(True, color='0.7', linestyle=':', linewidth=1)

In [ ]:
yo=pd.merge(news_rate,dr_score,left_index=True,right_index=True)
df6 = yo[['score','BMR']]
df6.cor()